# Construction of Datasets for Evaluation on S&P500

This notebook contains 6 sections:
1. Scraping the constituents and the relevant changes over the last T years
2. Create aggregate dataset
3. Modification of tickers (some are wrong)
4. API request with 'AlphaVantage' to get obtain data. Notice that an AlphaVantage API key is required.
5. Download SP500 index data with yfinance
6. Save relevant data to access it later from the models

## Dependencies

In [1]:
# Import date structuring
import datetime as dt
from datetime import datetime, timedelta, date

# Import required modules
import requests

# Import parser
from bs4 import BeautifulSoup

# Import .csv-handler
import csv

# Import data structure
import pandas as pd
import numpy as np

# Get data from Yahoo Finance
import yfinance as yf

# Import download bar
from tqdm import tqdm

# Import regex
import re

# Import Path to automate saving of data 
from pathlib import Path

# Import Alpha Vantage library
from EITP.Data.DataFetcher import AlphaVantageFinance
from EITP.Data.DataLoader import DataLoader

## 1. Scraping

We cannot make any quality guarantees, but for the purpose of performance testing it should suffice. The following Wikipedia page provides an extensive overview of the constituents and the historical changes to the index maintained by S&P500.

Link: https://en.wikipedia.org/wiki/List_of_S%26P_500_companies

We start by specifying the range of interest. This is limited by the amount of historical data that can be obtained through AlphaVantage.

In [2]:
startingYear = 1999
startingMonth = 1
startingDay = 1

startingDate = date(startingYear, startingMonth, startingDay)
endDate = date.today()

Then we specify the URL's to obtain the constituents.

In [ ]:
# Define the main URL
URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
 
# Get URL
page = requests.get(URL)

Once the content has been fetched with a HTTP request, we need to parse it. For this, we will use the BeautifulSoup library. This process is usually referred to as the actual scraping.

In [ ]:
# Scrape webpage
soup = BeautifulSoup(page.content, 'html.parser')

Then we wish to scrabe the table elements from the DOM.

In [ ]:
# Locate number of tables
len(soup.find_all('table'))

We can also scrape the tables individually by locating the right 'id'.

In [ ]:
tableConstituents = soup.find(id='constituents')
tableChanges = soup.find(id='changes')

Then we specify the column names in each table.

In [ ]:
columnNamesConstituents = [elem.text.strip() for elem in tableConstituents.find_all('th')]
columnNamesChanges = [elem.text.strip() for elem in tableChanges.find_all('th')]

The changes column does however have a different structure. Therefore, we remove 'Added' and 'Removed' and keep the order in mind later. In addition, we move 'Reason' to be the last element.

In [ ]:
columnNamesChanges = ['Date', 'AddedTicker', 'AddedSecurity', 'RemovedTicker', 'RemovedSecurity', 'Reason']

Next task is to parse the rows iteratively. From inspecting the webpage, it is seen that each row has the tag \<tr> and all the entries in the table are found with the tag \<td>.

### 1.1 Scrape Rows from Constituents Table

In [ ]:
rows = []
for row in tableConstituents.find_all('tr'):
    rowList = [elem.text.strip() for elem in row.find_all('td')]
    if rowList: rows.append(rowList)

In [ ]:
contituents = pd.DataFrame(rows, columns=columnNamesConstituents)

In [ ]:
# Convert the date column to date format
contituents["Date added"] = pd.to_datetime(contituents["Date added"], format="%Y-%m-%d", errors='coerce')

Some of the dates are missing. This is however not important as they are currently included in the S\&P500 index.

In [ ]:
# Then locate all the errors
def is_nat(x):
    return pd.isnull(x) and isinstance(x, pd._libs.tslibs.nattype.NaTType)

contituents[contituents["Date added"].apply(is_nat)]

### 1.2 Scrape Rows from Changes Table

In [ ]:
rows = []
for row in tableChanges.find_all('tr'):
    rowList = [elem.text.strip() for elem in row.find_all('td')]
    if rowList: rows.append(rowList)
        
# Create Pandas dataframe
changes = pd.DataFrame(rows, columns=columnNamesChanges)

# Convert the date column to date format
changes["Date"] = pd.to_datetime(changes["Date"], format="%B %d, %Y", errors='coerce')

## 2. Historical Constituents

We start by specifying the current constituents.

In [ ]:
# Define variables and add constituents today
tickers = contituents['Symbol'].to_list()
allRows = {'Date': [date.today().strftime('%Y-%m-%d')], 'Tickers': [','.join(tickers)], 'N': [len(tickers)]}
setTickers = set(tickers)
allTickers = set(tickers)

# Modify tickers according to changes
for DATE in np.flip(np.arange(startingDate, endDate, timedelta(days=1)).astype(datetime)):
    
    # Start by adding the date
    formattedDate = DATE.strftime("%Y-%m-%d")
    allRows['Date'].append(formattedDate)
    
    # The filter the changes
    match = changes[changes['Date'] == formattedDate]
    
    # If match, then carry out the following procedure
    if len(match) >= 1:
        
        # If AddedTicker contains a ticker, then remove it (as we traverse backwards in time)
        addedTicker = list(match['AddedTicker'].values)
        addedTicker = set([ticker for ticker in addedTicker if ticker != ''])
        setTickers = setTickers.difference(addedTicker)
        
        # If RemovedTicker contains a ticker, then add it (as we traverse backwards in time).
        # In addition, add the ticker to the allTickers.
        removedTicker = list(match['RemovedTicker'].values)
        removedTicker = set([ticker for ticker in removedTicker if ticker != ''])
        setTickers = setTickers.union(removedTicker)
        allTickers = allTickers.union(removedTicker)
        
    # Add to list
    allRows['Tickers'].append(','.join(list(setTickers)))
    
    # Add to list
    allRows['N'].append(len(setTickers))
        
# Aggregate the data
allTickers = list(allTickers)
historicalConstituents = pd.DataFrame.from_dict(allRows)

## 3. Ticker Modifications

It appears that Wikipedia does not have updates on all names changes. While it appears that some of the tickers have been listed under new ticker names, and thus can be obtained by fetching the updated ticker, they are removed as revising them manually requires too much time. This is of course a drawback with regards to the data quality. However, they are listed in a dictionary allowing any user to manually change the filtering.

In [ ]:
# Define a dictionary that handles all the missing matches in the AlphaVantage calls (daily)
dailyNameChanges = {"ADS": "BFH",  # Name change
               "HFC": "EXCLUDED",  # Does not exist in database
               "NYX": "ACQUIRED",  # Acquired by ICE Exchange
               "SMS": "ACQUIRED",  # Acquired by Siemens
               "FBHS": "REMOVED",  # Removed
               "GR": "ACQUIRED",   # Acquired
               "HPH": "BANKRUPCY", # Backrupcy
               "AV": "PRIVATE",    # Taken private 
               "TRB": "PRIVATE",   # Taken private 
               "ESV": "UNKNOWN",   # Unknown reason
               "SBL": "ACQUIRED",  # Acquired by Motorola
               "ACE": "EXR",       # Replaced due to acquisition
               "WIN": "UNKNOWN",   # Just not there
               "CBE": "UNKNOWN",   
               "FNM": "UNKNOWN",
               "QTRN": "UNKNOWN",
               "FRE": "UNKNOWN",
               "LEH": "UNKNOWN",
               "NOVL": "UNKNOWN",
               "BS": "UNKNOWN",
               "GLK": "UNKNOWN",
               "TYC": "UNKNOWN",
               "WFR": "UNKNOWN",
               "MEE": "UNKNOWN",
               "DJ": "UNKNOWN",
               "KSE": "UNKNOWN",
               "CFC": "UNKNOWN",
               "SLR": "UNKNOWN",
               "RX": "UNKNOWN",
               "CEPH": "UNKNOWN",
               "FRC": "UNKNOWN",
               "JCP": "UNKNOWN",
               "TLAB": "UNKNOWN",
               "ABK": "UNKNOWN",
               "PTV": "UNKNOWN",
               "EK": "UNKNOWN",
               "ABS": "UNKNOWN",
               "DF": "UNKNOWN",
               "CCE": "UNKNOWN",
               "SGP": "UNKNOWN",
               "LDW": "UNKNOWN",
               "GENZ": "UNKNOWN",
               "MOLX": "UNKNOWN",
               "BMC": "UNKNOWN",
               "TE": "UNKNOWN"
              }

# Define a dictionary that handles all the missing matches in the AlphaVantage calls (weekly)
weeklyNameChanges =  {"JCP": "JCP", 
                   "CFC": "CFC", 
                   "CEPH": "CEPH",
                   "NYX": "NYX",
                   "KSE": "KSE",
                   "NOVL": "NOVL",
                   "SBL": "SBL",
                   "ABS": "ABS",
                   "FNM": "FNM",
                   "FRC": "FRC",
                   "ABK": "ABK",
                   "BS": "BS",
                   "GLK": "GLK",
                   "TLAB": "TLAB",
                   "HPH": "HPH",
                   "GR": "GR",
                   "PTV": "PTV",
                   "LEH": "LEH",
                   "QTRN": "QTRN",
                   "FBHS": "FBHS",
                   "TYC": "TYC",
                   "AV": "AV",
                   "MOLX": "MOLX",
                   "ESV": "ESV",
                   "SGP": "SGP",
                   "MEE": "MEE",
                   "DJ": "DJ",
                   "SLR": "SLR",
                   "LDW": "LDW",
                   "HFC": "HFC",
                   "EK": "EK",
                   "GENZ": "GENZ",
                   "FRE": "FRE",
                   "ACE": "ACE",
                   "BMC": "BMC",
                   "CBE": "CBE",
                   "WFR": "WFR",
                   "RX": "RX",
                   "TRB": "TRB",
                   "CCE": "CCE",
                   "WIN": "WIN",
                   "TE": "TE",
                   "SMS": "SMS",
                   "DF": "DF"
                  }

# Then we filter the tickers
filteredTickersDaily = [ticker for ticker in allTickers if ticker not in dailyNameChanges]
filteredTickersWeekly = [ticker for ticker in allTickers if ticker not in weeklyNameChanges]

## 4. Call *AlphaVantage* to Fetch Prices

In this section, we use our custom AlphaVantage class to handle the communication with the external API. Afterwards, the class is used to construct the data sets containing the historical prices.

In [ ]:
# Create an instance of the class
dailyFinanceAPI = AlphaVantageFinance()
weeklyFinanceAPI = AlphaVantageFinance()

# Modify the tickers to fit the right format for the API (replace "." with "-")
modifiedTickersDaily, matchesWeekly, modifiedMatchesDaily = dailyFinanceAPI.modifyTickers(filteredTickersDaily, pattern=r'\b\w+\.\w+\b', replaceWith="-", replaceWhich=".")
modifiedTickersWeekly, matchesWeekly, modifiedMatchesWeekly = weeklyFinanceAPI.modifyTickers(filteredTickersWeekly, pattern=r'\b\w+\.\w+\b', replaceWith="-", replaceWhich=".")

We use the following call to fetch the data in a specified time interval. The following cell takes approximately 20 minutes to run.

In [ ]:
# Select from relevant interval (if none, then all available data is returned)
aggregateDailySP500 = dailyFinanceAPI.returnPricesInterval(symbols=modifiedTickersDaily, function="DAILY_ADJUSTED", sampleType="5. adjusted close", redownload=False)
aggregateWeeklySP500 = weeklyFinanceAPI.returnPricesInterval(symbols=modifiedTickersWeekly, function="WEEKLY_ADJUSTED", sampleType="5. adjusted close", redownload=False)

## 5. Download Historical Data for SP500

In this section, we download the index data for S&P500 (ticker: SPX) via Yahoo Finance. This is because Alpha Vantage does not support indices. 

In [ ]:
# Download daily data
dailySP500Index = yf.download(
    tickers = ["^GSPC"],
    start = min(aggregateDailySP500['Dates']),
    interval = "1d"
)['Adj Close']

dailySP500Index = pd.DataFrame({'Dates': dailySP500Index.index.values,'SPX-INDEX': dailySP500Index.values})

# Download weekly data
weeklySP500Index = yf.download(
    tickers = ["^GSPC"],
    start = min(aggregateWeeklySP500['Dates']),
    interval = "1wk"
)['Adj Close']

weeklySP500 = pd.DataFrame({'Dates': weeklySP500Index.index.values,'SPX-INDEX': weeklySP500Index.values})

Then we can join the index values of SPX with all the assets.

In [ ]:
completeDailySP500 = pd.merge(dailySP500Index, aggregateDailySP500, on='Dates', how="right")
completeWeeklySP500 = pd.merge(dailySP500Index, aggregateWeeklySP500, on='Dates', how="right")

## 6. Save Constituents, GICS Categories and Historical Data to .csv

In this section, we save the constructed dataframes as .csv-files to ease reusability.

In [ ]:
# Construct filepaths
filepath1 = Path('./Data/SP500/HistoricalConstituents.csv')  
filepath2 = Path('./Data/SP500/DailyHistoricalPrices.csv')  
filepath3 = Path('./Data/SP500/WeeklyHistoricalPrices.csv')
filepath4 = Path('./Data/SP500/ConstituentInformation.csv')

# Create necessary directories
filepath1.parent.mkdir(parents=True, exist_ok=True)

# Construct data sets
historicalConstituents.to_csv(filepath1, index=False)
completeDailySP500.to_csv(filepath2, index=False)
completeWeeklySP500.to_csv(filepath3, index=False)
contituents.to_csv(filepath4, index=False)

We test reading the constituents and create a dictionary that can be used later.

In [ ]:
filepath4 = Path('./Data/SP500/ConstituentInformation.csv')
test = pd.read_csv(filepath4)
industryMap = dict()
sectors = test['GICS Sector'].values
subIndustries = test['GICS Sector'].values
for idx, ticker in enumerate(test['Symbol'].values):
    industryMap[ticker] = {'Sector': sectors[idx],'Sub-Industry': subIndustries[idx]}

## 7. Creating a filtered data set with Data Loader

We initialize the class.

In [34]:
DL = DataLoader(path='./Data/')

Then we can save SP500RawFull.

In [66]:
# Load data from raw file
dailySP500RawFull = DL.SP500(freq="daily", intersect=False, startDate="1999-01-01", endDate=dt.datetime.today().strftime("%Y-%m-%d"))
dailyRawTickers = np.array(dailyStockData.columns[2:], dtype=str)

# Load data from raw file
weeklySP500RawFull = DL.SP500(freq="weekly", intersect=False, startDate="1999-01-01", endDate=dt.datetime.today().strftime("%Y-%m-%d"))
weeklySP500RawFull = weeklySP500RawFull.loc[:, dailySP500RawFull.columns]
weeklyRawTickers = dailyRawTickers

# Construct filepaths
filepath1 = Path('./Data/SP500/dailySP500RawFull.csv')
filepath2 = Path('./Data/SP500/weeklySP500RawFull.csv')  

# Create necessary directories
filepath1.parent.mkdir(parents=True, exist_ok=True)
filepath2.parent.mkdir(parents=True, exist_ok=True)

# Construct data sets
dailySP500RawFull.to_csv(filepath1, index=False)
weeklySP500RawFull.to_csv(filepath2, index=False)

Then we can save SP500RawIntersect.

In [79]:
# Load data from raw file
dailySP500RawIntersect = DL.SP500(freq="daily", intersect=True, startDate="1999-01-01", endDate=dt.datetime.today().strftime("%Y-%m-%d"))
dailyIntersectTickers = np.array(dailySP500RawIntersect.columns[2:], dtype=str)

# Load data from raw file
weeklySP500RawIntersect = DL.SP500(freq="weekly", intersect=True, startDate="1999-01-01", endDate=dt.datetime.today().strftime("%Y-%m-%d"))
weeklySP500RawIntersect = weeklySP500RawIntersect.loc[:, dailySP500RawIntersect.columns]
weeklyIntersectTickers = dailyIntersectTickers

# Construct filepaths
filepath1 = Path('./Data/SP500/dailySP500RawIntersect.csv')
filepath2 = Path('./Data/SP500/weeklySP500RawIntersect.csv')  

# Create necessary directories
filepath1.parent.mkdir(parents=True, exist_ok=True)
filepath2.parent.mkdir(parents=True, exist_ok=True)

# Construct data sets
dailySP500RawIntersect.to_csv(filepath1, index=False)
weeklySP500RawIntersect.to_csv(filepath2, index=False)

## Exclude GICS sub-industries

Then we try to exclude GICS sectors that we do not want to consider.

In [87]:
industryMap = DL.filterIndustries()

Suppose we want to exclude the following categories.

In [88]:
blacklist = ['Aerospace & Defense', 
             'Oil & Gas Equipment & Services'
             'Casinos & Gaming',
             'Integrated Oil & Gas',
             'Oil & Gas Exploration & Production',
             'Oil & Gas Refining & Marketing',
             'Tobacco',
             'Brewers',
             'Distillers & Vintners',
             'Oil & Gas Storage & Transportation']

Then we filter our tickers.

In [89]:
# Start with raw
filteredTickersRaw = DL.filterTickers(dailyRawTickers, blacklist)

# .. then intersect
filteredTickersIntersect = DL.filterTickers(dailyIntersectTickers, blacklist)

Then we modifiy the stock data to only include these. Due to inconsistency in the downloaded prices, the weekly dataset has an additional column. It is removed by considering the columns from the daily dataset.

In [105]:
# Start by creating it on raw
selector = list(dailySP500RawFull.columns[:2]) + filteredTickersRaw
dailySP500FilteredFull = dailySP500RawFull.loc[:,selector]
weeklySP500FilteredFull = weeklySP500RawFull.loc[:,selector]

# .. then create it on filtered
selector = list(dailySP500RawIntersect.columns[:2]) + filteredTickersIntersect
dailySP500FilteredIntersect = dailySP500RawIntersect.loc[:,selector]
weeklySP500FilteredIntersect = weeklySP500RawIntersect.loc[:,selector]

Now the stock data holds all those assets not blacklisted. These can be fed directly to the model.

In [106]:
# Construct filepaths
filepath1 = Path('./Data/SP500/dailySP500FilteredFull.csv')
filepath2 = Path('./Data/SP500/weeklySP500FilteredFull.csv')  
filepath3 = Path('./Data/SP500/dailySP500FilteredIntersect.csv')
filepath4 = Path('./Data/SP500/weeklySP500FilteredIntersect.csv') 

# Create necessary directories
filepath1.parent.mkdir(parents=True, exist_ok=True)
filepath2.parent.mkdir(parents=True, exist_ok=True)
filepath3.parent.mkdir(parents=True, exist_ok=True)
filepath4.parent.mkdir(parents=True, exist_ok=True)

# Construct data sets
dailySP500FilteredFull.to_csv(filepath1, index=False)
weeklySP500FilteredFull.to_csv(filepath2, index=False)
dailySP500FilteredIntersect.to_csv(filepath3, index=False)
weeklySP500FilteredIntersect.to_csv(filepath4, index=False)